In [ ]:
import gradio as gr
import datetime

# --- Data ---
food_calories = {
    "Apple (1 medium)": 95,
    "Banana (1 medium)": 105,
    "Boiled Egg (1 large)": 70,
    "White Rice (1 cup cooked)": 200,
    "Brown Rice (1 cup cooked)": 215,
    "Chicken Breast (100g, cooked)": 165,
    "Beef (100g, cooked)": 250,
    "Salmon (100g, cooked)": 210,
    "Tofu (100g, firm)": 145,
    "Broccoli (1 cup cooked)": 55,
    "Carrot (1 medium)": 25,
    "Potato (1 medium, boiled)": 160,
    "Sweet Potato (1 medium, baked)": 100,
    "Almonds (28g / ~23 pcs)": 160,
    "Peanut Butter (1 tbsp)": 95,
    "Milk (1 cup, whole)": 150,
    "Yogurt (plain, 1 cup)": 100,
    "Cheddar Cheese (28g / 1 oz)": 115,
    "Bread (1 slice, white)": 80,
    "Pasta (1 cup cooked)": 220,
    "Pad Thai (Chicken)": 450,
    "Green Curry with Chicken and Rice": 600,
    "Tom Yum Goong": 250,
    "Massaman Curry with Beef and Rice": 700,
    "Pad Kra Pao with Rice and Egg": 600,
    "Som Tum": 150,
    "Khao Pad (Chicken Fried Rice)": 550,
    "Khao Soi": 650,
    "Larb (Chicken)": 250,
    "Panang Curry with Chicken and Rice": 600,
    "Satay (3 Chicken Skewers)": 200,
    "Thai Spring Rolls (2 pcs)": 250,
    "Moo Ping (2 Skewers)": 220,
    "Sticky Rice with Mango": 400,
    "Thai Coconut Ice Cream (1 scoop)": 300,
    "Yum Woon Sen (Glass Noodle Salad)": 200,
    "Gai Tod (Fried Chicken)": 300,
    "Khanom Krok (Coconut Pancakes, 4 pcs)": 250,
    "Pad See Ew (Stir-Fried Flat Noodles with Pork)": 550,
    "Gaeng Daeng (Red Curry with Chicken and Rice)": 600,
}

activity_levels = {
    "Sedentary (little or no exercise)": 1.2,
    "Lightly active (light exercise/sports 1-3 days/week)": 1.375,
    "Moderately active (moderate exercise/sports 3-5 days/week)": 1.55,
    "Very active (hard exercise/sports 6-7 days a week)": 1.725,
    "Super active (very hard exercise/physical job)": 1.9,
}

# --- Custom CSS for Styling ---
custom_css = """
:root {
    --primary-hue: 318deg; /* For #582C4D */
    --primary-hue-dark: 318deg;
    --primary-hue-light: 318deg;
    --text-color: #ECE2D0;
    --body-background-fill: #BE92A2;
    --background-fill-primary: #D5B9B2;
    --background-fill-secondary: #D5B9B2;
    --button-primary-background-fill: #582C4D;
    --button-primary-background-fill-hover: #6a3a5c;
    --button-primary-text-color: #ECE2D0;
    --slider-color: #582C4D;
}
body {
    background-color: var(--body-background-fill);
    color: var(--text-color);
}
.gradio-container {
    background: var(--body-background-fill);
}
.gr-panel {
    background-color: var(--background-fill-primary) !important;
}
.gr-box {
     background-color: var(--background-fill-primary) !important;
}
.gr-input {
    background-color: #ffffff; /* White background for better readability */
}
h1, h2, h3, label, p, .gr-button {
    color: var(--text-color) !important;
}
footer {
    display: none !important;
}
"""

# --- Tab 1: BMI Calculator Logic ---
def calculate_bmi(height, weight, height_unit, weight_unit):
    if not height or not weight:
        return "Please enter height and weight.", "", ""

    # Convert to metric
    if height_unit == "inches":
        height_m = height * 0.0254
    else:  # cm
        height_m = height / 100

    if weight_unit == "lbs":
        weight_kg = weight * 0.453592
    else:  # kg
        weight_kg = weight

    if height_m == 0:
        return "Height cannot be zero.", "", ""

    bmi = weight_kg / (height_m ** 2)
    bmi_rounded = round(bmi, 1)

    # Determine category
    if bmi < 18.5:
        category = "Underweight"
        color = "lightblue"
    elif 18.5 <= bmi < 25:
        category = "Normal weight"
        color = "lightgreen"
    elif 25 <= bmi < 30:
        category = "Overweight"
        color = "gold"
    else:
        category = "Obese"
        color = "lightcoral"

    result_html = f"""
    <div style='text-align: center; padding: 20px; border-radius: 10px; background-color: {color};'>
        <p style='font-size: 1.2em; color: #000;'>Your BMI is</p>
        <p style='font-size: 2.5em; font-weight: bold; color: #000;'>{bmi_rounded}</p>
        <p style='font-size: 1.2em; color: #000;'>Category: {category}</p>
    </div>
    """
    return f"{bmi_rounded}", category, result_html

# --- Tab 2: BMR & TDEE Calculator Logic ---
def calculate_tdee(age, gender, height, weight, activity_level):
    if not all([age, gender, height, weight, activity_level]):
        return "Please fill all fields.", "", ""

    # Harris-Benedict Equation for BMR
    if gender == "Male":
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:  # Female
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)

    tdee = bmr * activity_levels[activity_level]

    bmr_rounded = round(bmr)
    tdee_rounded = round(tdee)

    goals = (
        f"**Maintain weight:** {tdee_rounded} kcal/day\n"
        f"**Mild weight loss (0.25 kg/week):** {tdee_rounded - 250} kcal/day\n"
        f"**Weight loss (0.5 kg/week):** {tdee_rounded - 500} kcal/day\n"
        f"**Mild weight gain (0.25 kg/week):** {tdee_rounded + 250} kcal/day\n"
        f"**Weight gain (0.5 kg/week):** {tdee_rounded + 500} kcal/day\n"
    )
    return f"{bmr_rounded} kcal/day", f"{tdee_rounded} kcal/day", goals

# --- Tab 3: Food Tracker Logic ---
def add_food_to_log(food_item, manual_calories, current_total):
    calories_to_add = 0
    if food_item and food_item in food_calories:
        calories_to_add = food_calories[food_item]
    elif manual_calories > 0:
        calories_to_add = manual_calories

    new_total = current_total + calories_to_add
    return new_total, f"Daily Total: {new_total} kcal"

def reset_log():
    return 0, "Daily Total: 0 kcal"

def compare_calories(daily_total, tdee_input):
    if not tdee_input:
        return "Enter your TDEE from Tab 2 to see a comparison."

    try:
        tdee_value = float(str(tdee_input).split()[0])
        difference = daily_total - tdee_value
        if difference > 0:
            return f"You are {abs(round(difference))} kcal OVER your daily goal."
        elif difference < 0:
            return f"You are {abs(round(difference))} kcal UNDER your daily goal."
        else:
            return "You have met your daily goal exactly!"
    except (ValueError, IndexError):
        return "Invalid TDEE format. Please enter a number (e.g., '2000')."

# --- Gradio App ---
with gr.Blocks(css=custom_css, title="Health Calculator") as demo:
    gr.Markdown("# Modern Health & Fitness Calculator")

    # Shared state for TDEE and calorie log
    tdee_state = gr.State(0)
    calorie_log_total = gr.State(0)

    with gr.Tabs():
        # --- Tab 1: BMI Calculator ---
        with gr.TabItem("BMI Calculator"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("## Calculate Your Body Mass Index")
                    height_input = gr.Number(label="Height", value=170)
                    height_unit_input = gr.Radio(["cm", "inches"], label="Height Unit", value="cm")
                    weight_input = gr.Number(label="Weight", value=65)
                    weight_unit_input = gr.Radio(["kg", "lbs"], label="Weight Unit", value="kg")
                    bmi_btn = gr.Button("Calculate BMI")
                with gr.Column():
                    gr.Markdown("## Your Result")
                    bmi_output_html = gr.HTML()

            # Hidden outputs for potential future use
            bmi_output_value = gr.Textbox(label="BMI Value", visible=False)
            bmi_output_category = gr.Textbox(label="BMI Category", visible=False)

        # --- Tab 2: Daily Metabolic Rate Calculator ---
        with gr.TabItem("Daily Calorie Needs (BMR & TDEE)"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("## Calculate Your Daily Energy Expenditure")
                    age_input = gr.Slider(2, 100, value=30, step=1, label="Age")
                    gender_input = gr.Radio(["Male", "Female"], label="Gender", value="Male")
                    bmr_height_input = gr.Number(label="Height (cm)", value=170)
                    bmr_weight_input = gr.Number(label="Weight (kg)", value=65)
                    activity_level_input = gr.Dropdown(
                        list(activity_levels.keys()),
                        label="Activity Level",
                        value="Lightly active (light exercise/sports 1-3 days/week)"
                    )
                    tdee_btn = gr.Button("Calculate Calorie Needs")
                with gr.Column():
                    gr.Markdown("## Your Results")
                    bmr_output = gr.Textbox(label="Basal Metabolic Rate (BMR)")
                    tdee_output = gr.Textbox(label="Total Daily Energy Expenditure (TDEE)")
                    gr.Markdown("### Calorie Goals for Weight Management")
                    goals_output = gr.Markdown()

        # --- Tab 3: Food Tracker ---
        with gr.TabItem("Food Calorie Tracker"):
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("## Log Your Daily Food Intake")
                    tdee_input_for_tracker = gr.Number(label="Enter Your TDEE (from Tab 2)", info="This helps compare your intake to your goal.")
                    food_dropdown = gr.Dropdown(list(food_calories.keys()), label="Select a Food Item")
                    manual_calories_input = gr.Number(label="Or Enter Calories Manually", value=0)
                    add_food_btn = gr.Button("Add to Daily Log")
                    reset_log_btn = gr.Button("Reset Daily Log")
                with gr.Column(scale=1):
                    gr.Markdown("## Today's Summary")
                    daily_total_display = gr.Label("Daily Total: 0 kcal")
                    comparison_output = gr.Textbox(label="Comparison to TDEE Goal", interactive=False)

    # --- Event Handlers ---
    bmi_btn.click(
        fn=calculate_bmi,
        inputs=[height_input, weight_input, height_unit_input, weight_unit_input],
        outputs=[bmi_output_value, bmi_output_category, bmi_output_html]
    )

    tdee_btn.click(
        fn=calculate_tdee,
        inputs=[age_input, gender_input, bmr_height_input, bmr_weight_input, activity_level_input],
        outputs=[bmr_output, tdee_output, goals_output]
    )

    add_food_btn.click(
        fn=add_food_to_log,
        inputs=[food_dropdown, manual_calories_input, calorie_log_total],
        outputs=[calorie_log_total, daily_total_display]
    ).then(
        fn=compare_calories,
        inputs=[calorie_log_total, tdee_input_for_tracker],
        outputs=[comparison_output]
    )

    tdee_input_for_tracker.change(
        fn=compare_calories,
        inputs=[calorie_log_total, tdee_input_for_tracker],
        outputs=[comparison_output]
    )

    reset_log_btn.click(
        fn=reset_log,
        inputs=[],
        outputs=[calorie_log_total, daily_total_display]
    ).then(
        fn=compare_calories,
        inputs=[calorie_log_total, tdee_input_for_tracker],
        outputs=[comparison_output]
    )


if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a256d02f23c14c7af3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
